In [227]:
import numpy as np
import pandas as pd
import json
import os
import warnings
warnings.filterwarnings("ignore")

# Merge full data category

In [125]:
path_category = "C:/Users/lydtm/Downloads/Hasaki/data_processing/data_category/"

files = os.listdir(path_category)
df_full = pd.DataFrame()
for f in files:
    if (f[-4:] == 'xlsx') and ("result_full_data" not in f) and (f != "quantity.xlsx"): # (f != "result_full_data.xlsx")
        data = pd.read_excel(path_category + f, dtype={'item_id': 'str', 'item_product': 'Int64', 'item_comment_number': 'Int64', 'item_bought': 'str'})
        data = data.drop(columns=['Column1', 'item_variant'])
        data['category'] = f[7:-5]
        df_full = df_full.append(data)

In [126]:
df_full = df_full.drop_duplicates()

In [127]:
df_full = df_full[~df_full['item_name'].isna()]
df_full = df_full[~df_full['item_brand'].isna()]
df_full = df_full[~df_full['item_id'].isna()]

In [128]:
df_full['item_bought'] = df_full['item_bought'].apply(lambda s: int(str(s).replace(".", "")) if isinstance(s, str) else 0)

In [129]:
def convert_value(value):
    if isinstance(value, str):
        if '%' in value:
            value = float(value[:-1]) / 100 
        else:
            value = float(value)
    return value

df_full['item_star'] = df_full['item_star'].apply(convert_value)
df_full['item_discount'] = df_full['item_discount'].apply(convert_value)

In [130]:
df_full['item_discount'] = df_full['item_discount'].fillna(0.0)

In [131]:
def parse_price(s):
    try:
        return float(s[:-2].replace(".", ""))
    except (ValueError, TypeError):
        return float('nan')

df_full['item_price_previous'] = df_full['item_price_previous'].apply(parse_price)

In [132]:
df_full['item_price_previous'] = df_full.apply(lambda row: row['item_price'] if row['item_discount'] == 0.0 else row['item_price_previous'], axis=1)

In [133]:
df_full['count_null'] = df_full.isnull().sum(axis=1)
df_full = df_full.sort_values('count_null')
df_full = df_full.drop_duplicates(subset=['item_id', 'date'], keep='first')
df_full = df_full.drop(columns='count_null')

In [134]:
df_full['item_star'] = df_full['item_star'] * 5

In [277]:
df_full.sort_values(['item_id', 'date'], inplace=True)
df_full['bought_by_day'] = df_full.groupby('item_id')['item_bought'].diff()

In [273]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226916 entries, 0 to 226915
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   item_name            226916 non-null  object        
 1   item_brand           226916 non-null  object        
 2   item_id              226916 non-null  object        
 3   item_product         226916 non-null  int64         
 4   item_price           226916 non-null  int64         
 5   item_bought          226916 non-null  int64         
 6   item_discount        226916 non-null  float64       
 7   item_price_previous  226916 non-null  int64         
 8   item_star            129134 non-null  float64       
 9   item_comment_number  129134 non-null  float64       
 10  date                 226916 non-null  datetime64[ns]
 11  category             226916 non-null  object        
 12  bought_by_day        218576 non-null  float64       
dtypes: datetime64[

In [274]:
df_full.isnull().sum() / df_full.shape[0]

item_name              0.000000
item_brand             0.000000
item_id                0.000000
item_product           0.000000
item_price             0.000000
item_bought            0.000000
item_discount          0.000000
item_price_previous    0.000000
item_star              0.430917
item_comment_number    0.430917
date                   0.000000
category               0.000000
bought_by_day          0.036754
dtype: float64

In [275]:
df_full.head(5)

,item_name,item_brand,item_id,item_product,item_price,item_bought,item_discount,item_price_previous,item_star,item_comment_number,date,category,bought_by_day
0,Bông Tẩy Trang Byphasse 120 Miếng,BYPHASSE,100020010,3212,35000,34,0.53,75000,4.0,1.0,2023-08-30,trang_diem,NaN
1,Bông Tẩy Trang Byphasse 120 Miếng,BYPHASSE,100020010,3212,35000,34,0.53,75000,4.0,1.0,2023-08-31,trang_diem,0.0
2,Bông Tẩy Trang Byphasse 120 Miếng,BYPHASSE,100020010,3212,35000,34,0.53,75000,4.0,1.0,2023-09-01,trang_diem,0.0
3,Bông Tẩy Trang Byphasse 120 Miếng,BYPHASSE,100020010,3212,35000,34,0.53,75000,4.0,1.0,2023-09-02,trang_diem,0.0
4,Bông Tẩy Trang Byphasse 120 Miếng,BYPHASSE,100020010,3212,35000,34,0.53,75000,4.0,1.0,2023-09-03,trang_diem,0.0


In [276]:
df_full.describe()

,item_product,item_price,item_bought,item_discount,item_price_previous,item_star,item_comment_number,bought_by_day
count,226916.000000,2.269160e+05,226916.000000,226916.000000,2.269160e+05,129134.000000,129134.000000,218576.000000
mean,87797.097446,2.418716e+05,47.873526,0.227543,3.184731e+05,4.565338,12.667903,-0.010697
std,29206.306398,3.231043e+05,204.884278,0.163053,4.101054e+05,0.594693,32.900575,51.313942
min,843.000000,4.500000e+03,0.000000,0.000000,4.500000e+03,1.000000,1.000000,-5510.000000
25%,86287.000000,7.900000e+04,0.000000,0.110000,1.000000e+05,4.300000,2.000000,0.000000
50%,98185.000000,1.580000e+05,4.000000,0.210000,1.990000e+05,4.800000,4.000000,0.000000
75%,105055.000000,2.820000e+05,25.000000,0.320000,3.800000e+05,5.000000,10.000000,0.000000
max,112723.000000,3.933000e+06,6055.000000,0.910000,4.990000e+06,5.000000,553.000000,5510.000000


In [139]:
# df_full.to_excel(path_category + "result_full_data.xlsx", index=False, encoding='utf-8')

## Fix lỗi (1 item_name - n item_id) và (1 item_id - n item_name)

In [249]:
df_full = pd.read_excel(path_category + "result_full_data.xlsx", dtype={'item_id': 'str'})

In [250]:
df_full.isnull().sum()

item_name                  0
item_brand                 0
item_id                    0
item_product               0
item_price                 0
item_bought                0
item_discount              0
item_price_previous        0
item_star              97782
item_comment_number    97782
date                       0
category                   0
bought_by_day           8340
dtype: int64

In [252]:
df_full[['item_name', 'item_id']].nunique()

item_name    8456
item_id      8340
dtype: int64

In [253]:
### 1 item_id - n item_name

# Đổi giá trị cột "item_brand" sang chữ hoa
df_full['item_brand'] = df_full['item_brand'].str.upper()

# Tìm các dòng với item_id giống nhau nhưng item_brand khác nhau
# duplicated_rows = tmp.groupby('item_id').filter(lambda x: x['item_brand'].nunique() > 1).drop_duplicates(subset=['item_brand', 'item_id'])
# duplicated_rows

df_full['item_name'] = df_full.groupby('item_id')['item_name'].transform('first')

In [254]:
# Check
df_full.groupby('item_id').filter(lambda x: x['item_brand'].nunique() > 1)#.drop_duplicates(subset=['item_brand', 'item_id'])

,item_name,item_brand,item_id,item_product,item_price,item_bought,item_discount,item_price_previous,item_star,item_comment_number,date,category,bought_by_day


In [256]:
# Check
df_full.groupby('item_name').filter(lambda x: x['item_id'].nunique() > 1).drop_duplicates(subset=['item_name', 'item_id']) # bỏ drop 

,item_name,item_brand,item_id,item_product,item_price,item_bought,item_discount,item_price_previous,item_star,item_comment_number,date,category,bought_by_day


In [257]:
# Check
df_full.groupby('item_id').filter(lambda x: x['item_name'].nunique() > 1).drop_duplicates(subset=['item_name', 'item_id'])

,item_name,item_brand,item_id,item_product,item_price,item_bought,item_discount,item_price_previous,item_star,item_comment_number,date,category,bought_by_day


# Tạo thêm cột số lượng sản phẩm được mua trong 1 tháng ứng với từng sản phẩm

In [258]:
# Tạo thêm cột số lượng sản phẩm được mua trong 1 tháng ứng với từng sản phẩm
quantity_diff = df_full.groupby(['item_id', 'item_name'])['item_bought'].max() - df_full.groupby(['item_id', 'item_name'])['item_bought'].min()
df_quantity = quantity_diff.reset_index().rename(columns={'item_bought': 'quantity'})
df_quantity = df_quantity.drop_duplicates(subset=['item_id'])
df_quantity

,item_id,item_name,quantity
0,100020010,Bông Tẩy Trang Byphasse 120 Miếng,3
1,100020011,Kem Tẩy Lông Byphasse Chiết Xuất Tơ Tằm 125ml,0
2,100070011,Kem Dưỡng Thể Paula's Choice giảm viêm lỗ chân...,9
3,100070047,Sữa Dưỡng Thể Nivea Dưỡng Ẩm Da Chuyên Sâu 250ml,0
4,100070048,Serum Dưỡng Thể Nivea Sáng Da Ban Đêm 180ml,8
...,...,...,...
8335,422215227,[HSD 04/2024] Mặt Nạ Freeman Than Hoạt Tính & ...,0
8336,422215235,[HSD 04/2024] Kem Dưỡng Laneige Cung Cấp Nước ...,0
8337,422215238,[HSD 06/2024] Lăn Khử Mùi Deonatulle Đá Khoáng...,0
8338,422215239,[HSD 05/2024] Viên Uống Nature'Bounty,0


In [259]:
df_quantity.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8340 entries, 0 to 8339
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   item_id    8340 non-null   object
 1   item_name  8340 non-null   object
 2   quantity   8340 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 260.6+ KB


In [260]:
# df_quantity.to_excel(path_category + "quantity.xlsx", index=False, encoding='utf-8')

In [261]:
# tmp = pd.read_excel(path_category + "quantity.xlsx", dtype={'item_id': 'str'})
# tmp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8340 entries, 0 to 8339
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   item_id    8340 non-null   object
 1   item_name  8340 non-null   object
 2   quantity   8340 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 195.6+ KB


In [263]:
# tmp.head(5)

,item_id,item_name,quantity
0,100020010,Bông Tẩy Trang Byphasse 120 Miếng,3
1,100020011,Kem Tẩy Lông Byphasse Chiết Xuất Tơ Tằm 125ml,0
2,100070011,Kem Dưỡng Thể Paula's Choice giảm viêm lỗ chân...,9
3,100070047,Sữa Dưỡng Thể Nivea Dưỡng Ẩm Da Chuyên Sâu 250ml,0
4,100070048,Serum Dưỡng Thể Nivea Sáng Da Ban Đêm 180ml,8


In [270]:
# tmp.describe()

,quantity
count,8340.000000
mean,6.499520
std,115.329438
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,5510.000000


# Merge data detail 

In [ ]:
path_detail = "C:/Users/lydtm/Downloads/Hasaki/data_processing/data_detail/"

files = os.listdir(path_detail)
df_detail = pd.DataFrame()
for f in files:
    if (f[-4:] == 'xlsx') and (f != "information_full_data.xlsx"):
        data = pd.read_excel(path_detail + f, dtype={'item_id': 'str'})
        # data = data.drop(columns=['Unnamed: 0'])
        data['category'] = f[12:-5]
        df_detail = df_detail.append(data)

In [ ]:
df_detail = df_detail.drop(columns=['Unnamed: 0'])

In [ ]:
df_detail = df_detail.drop_duplicates()

In [ ]:
df_detail = df_detail[~df_detail['item_brand'].isna()]

In [ ]:
df_detail.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8037 entries, 0 to 1759
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   item_name      8037 non-null   object
 1   item_brand     8037 non-null   object
 2   item_id        8037 non-null   object
 3   item_thong_so  8037 non-null   object
 4   category       8037 non-null   object
dtypes: object(5)
memory usage: 376.7+ KB


In [ ]:
df_detail

,item_name,item_brand,item_id,item_thong_so,category
0,Dung Dịch Vệ Sinh Phụ Nữ Femfresh Cao Cấp Hằng...,Femfresh,422205453,"{'Barcode': '5010724525937', 'Thương Hiệu': 'F...",ca_nhan
1,Dung Dịch Vệ Sinh Phụ Nữ Lactacyd Ngăn Mùi 24H...,Lactacyd,301500006,"{'Barcode': '', 'Thương Hiệu': 'Lactacyd', 'Xu...",ca_nhan
2,Combo 2 Nước Súc Miệng Listerine Hơi Thở Thơm ...,Listerine,294100026,"{'Barcode': '', 'Thương Hiệu': 'Listerine', 'X...",ca_nhan
3,Nước Súc Miệng Propolinse Pure Chiết Xuất Sáp ...,Propolinse,422201738,"{'Barcode': '4966680247004', 'Thương Hiệu': 'P...",ca_nhan
4,Kem Đánh Răng Eucryl Làm Trắng Răng Hương Bạc ...,Eucryl,372000002,"{'Barcode': '5011309895513', 'Thương Hiệu': 'E...",ca_nhan
...,...,...,...,...,...
1755,Son Kem Lì Aprilskin #3 Chilli Warming Màu Đỏ ...,Aprilskin,422208874,"{'Barcode': '8809557519865', 'Thương Hiệu': 'A...",trang_diem
1756,Son Artdeco Perfect Color Màu 909 Watermelon P...,Artdeco,422209883,"{'Barcode': '4052136093483', 'Thương Hiệu': 'A...",trang_diem
1757,Chì Kẻ Chân Mày Artdeco Designer Màu 2 Dark 1g,Artdeco,422209749,"{'Barcode': '4019674028124', 'Thương Hiệu': 'A...",trang_diem
1758,Bút Che Khuyết Điểm Artdeco Màu 3 Peach 1.8ml,Artdeco,422209773,"{'Barcode': '4019674049730', 'Thương Hiệu': 'A...",trang_diem


In [ ]:
df_detail.head(1)

,item_name,item_brand,item_id,item_thong_so,category
0,Dung Dịch Vệ Sinh Phụ Nữ Femfresh Cao Cấp Hằng...,Femfresh,422205453,"{'Barcode': '5010724525937', 'Thương Hiệu': 'F...",ca_nhan


In [ ]:
df_detail['item_thong_so'] = df_detail['item_thong_so'].str.replace("'", '"')

In [ ]:
brand_dict = {'Angel"s Liquid': "Angel's Liquid", 
         'Paula"s Choice': "Paula's Choice", 
         'Tesori d"Oriente': "Tesori d'Oriente", 
         'L"Ovite': "L'Ovite", 
         'J"White': "J'White",
         'It"S SKIN': "It'S SKIN",
         'L"Oreal': "L'Oreal",
         'POND"S': "POND'S",
         'Kiehl"s': "Kiehl's",
         'I"m from': "I'm from",
         'So"Natural': "So'Natural",
         'Shina"S': "Shina'S",
         'O"Lysee': "O'Lysee",
         'AGE 20"s': "AGE 20's",
         'NATURE"S BOUNTY': "NATURE'S BOUNTY"}

In [ ]:
# Function to replace brand names
def replace_brand_name(json_string):
    for key, value in brand_dict.items():
        json_string = json_string.replace(key, value)
    return json_string

# Apply brand name replacement to the 'item_thong_so' column
df_detail['item_thong_so'] = df_detail['item_thong_so'].apply(replace_brand_name)

In [ ]:
# parse_data = []
# failed_data = []
# cnt = 0
# for json_string in df_detail['item_thong_so']:
#     try:
#         parse_data.append(json.loads(json_string))
#     except json.JSONDecodeError as e:
#         print(cnt, json_string)
#         print("JSON decoding error:", e)
#         failed_data.append(json_string)
#     cnt += 1

In [ ]:
df_detail = pd.concat([df_detail.drop('item_thong_so', axis=1),
                      df_detail['item_thong_so'].apply(lambda x: pd.Series(json.loads(x)))], axis=1)

In [ ]:
df_detail

,item_name,item_brand,item_id,category,Barcode,Thương Hiệu,Xuất xứ thương hiệu,Nơi sản xuất,Đặc Tính,Giới tính,Loại da,Vấn đề về da
0,Dung Dịch Vệ Sinh Phụ Nữ Femfresh Cao Cấp Hằng...,Femfresh,422205453,ca_nhan,5010724525937,Femfresh,Anh,United Kingdom,Có Mùi Hương,Nữ,NaN,NaN
1,Dung Dịch Vệ Sinh Phụ Nữ Lactacyd Ngăn Mùi 24H...,Lactacyd,301500006,ca_nhan,,Lactacyd,Pháp,Vietnam,Có Mùi Hương,Nữ,NaN,NaN
2,Combo 2 Nước Súc Miệng Listerine Hơi Thở Thơm ...,Listerine,294100026,ca_nhan,,Listerine,Mỹ,,Vị Cay,Nam & nữ,NaN,NaN
3,Nước Súc Miệng Propolinse Pure Chiết Xuất Sáp ...,Propolinse,422201738,ca_nhan,4966680247004,Propolinse,Nhật Bản,Korea,Vị Cay,Nam & nữ,NaN,NaN
4,Kem Đánh Răng Eucryl Làm Trắng Răng Hương Bạc ...,Eucryl,372000002,ca_nhan,5011309895513,Eucryl,Anh,United Kingdom,Có Mùi Hương,Nam & nữ,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1755,Son Kem Lì Aprilskin #3 Chilli Warming Màu Đỏ ...,Aprilskin,422208874,trang_diem,8809557519865,Aprilskin,Hàn Quốc,Korea,Dạng Lì,Nam & nữ,NaN,NaN
1756,Son Artdeco Perfect Color Màu 909 Watermelon P...,Artdeco,422209883,trang_diem,4052136093483,Artdeco,Ðức,Germany,NaN,Nam & nữ,NaN,NaN
1757,Chì Kẻ Chân Mày Artdeco Designer Màu 2 Dark 1g,Artdeco,422209749,trang_diem,4019674028124,Artdeco,Ðức,Germany,Có Màu,Nam & nữ,NaN,NaN
1758,Bút Che Khuyết Điểm Artdeco Màu 3 Peach 1.8ml,Artdeco,422209773,trang_diem,4019674049730,Artdeco,Ðức,Germany,NaN,Nam & nữ,Da thường/ Mọi loại da,NaN


In [ ]:
df_detail = df_detail.rename(columns={
            'Barcode': 'barcode',
            'Thương Hiệu': 'thuong_hieu', 
            'Xuất xứ thương hiệu': 'xuat_xu_thuong_hieu', 
            'Nơi sản xuất': 'noi_san_xuat', 
            'Đặc Tính': 'dac_tinh',
            'Giới tính': 'gioi_tinh', 
            'Loại da': 'loai_da', 
            'Vấn đề về da': 'van_de_ve_da'
        })

In [ ]:
df_detail[['barcode', 'thuong_hieu', 'xuat_xu_thuong_hieu', 'noi_san_xuat', 'dac_tinh', 'gioi_tinh', 'loai_da', 'van_de_ve_da']] = df_detail[['barcode', 'thuong_hieu', 'xuat_xu_thuong_hieu', 'noi_san_xuat', 'dac_tinh', 'gioi_tinh', 'loai_da', 'van_de_ve_da']].replace('', np.nan)

In [ ]:
df_detail.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8037 entries, 0 to 1759
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   item_name            8037 non-null   object
 1   item_brand           8037 non-null   object
 2   item_id              8037 non-null   object
 3   category             8037 non-null   object
 4   barcode              6376 non-null   object
 5   thuong_hieu          8037 non-null   object
 6   xuat_xu_thuong_hieu  8004 non-null   object
 7   noi_san_xuat         5419 non-null   object
 8   dac_tinh             4010 non-null   object
 9   gioi_tinh            6552 non-null   object
 10  loai_da              3867 non-null   object
 11  van_de_ve_da         3238 non-null   object
dtypes: object(12)
memory usage: 816.3+ KB


In [ ]:
df_detail.isnull().sum() / df_detail.shape[0]

item_name              0.000000
item_brand             0.000000
item_id                0.000000
category               0.000000
barcode                0.206669
thuong_hieu            0.000000
xuat_xu_thuong_hieu    0.004106
noi_san_xuat           0.325743
dac_tinh               0.501058
gioi_tinh              0.184770
loai_da                0.518850
van_de_ve_da           0.597113
dtype: float64

In [ ]:
df_detail = df_detail.drop(columns=['barcode', 'dac_tinh', 'loai_da', 'van_de_ve_da', 'noi_san_xuat', 'thuong_hieu'])

In [ ]:
df_detail.isnull().sum()

item_name                 0
item_brand                0
item_id                   0
category                  0
xuat_xu_thuong_hieu      33
gioi_tinh              1485
dtype: int64

In [ ]:
df_detail

,item_name,item_brand,item_id,category,xuat_xu_thuong_hieu,gioi_tinh
0,Dung Dịch Vệ Sinh Phụ Nữ Femfresh Cao Cấp Hằng...,Femfresh,422205453,ca_nhan,Anh,Nữ
1,Dung Dịch Vệ Sinh Phụ Nữ Lactacyd Ngăn Mùi 24H...,Lactacyd,301500006,ca_nhan,Pháp,Nữ
2,Combo 2 Nước Súc Miệng Listerine Hơi Thở Thơm ...,Listerine,294100026,ca_nhan,Mỹ,Nam & nữ
3,Nước Súc Miệng Propolinse Pure Chiết Xuất Sáp ...,Propolinse,422201738,ca_nhan,Nhật Bản,Nam & nữ
4,Kem Đánh Răng Eucryl Làm Trắng Răng Hương Bạc ...,Eucryl,372000002,ca_nhan,Anh,Nam & nữ
...,...,...,...,...,...,...
1755,Son Kem Lì Aprilskin #3 Chilli Warming Màu Đỏ ...,Aprilskin,422208874,trang_diem,Hàn Quốc,Nam & nữ
1756,Son Artdeco Perfect Color Màu 909 Watermelon P...,Artdeco,422209883,trang_diem,Ðức,Nam & nữ
1757,Chì Kẻ Chân Mày Artdeco Designer Màu 2 Dark 1g,Artdeco,422209749,trang_diem,Ðức,Nam & nữ
1758,Bút Che Khuyết Điểm Artdeco Màu 3 Peach 1.8ml,Artdeco,422209773,trang_diem,Ðức,Nam & nữ


In [ ]:
df_detail = df_detail.drop_duplicates()

In [ ]:
df_detail['count_null'] = df_detail.isnull().sum(axis=1)
df_detail = df_detail.sort_values('count_null')
df_detail = df_detail.drop_duplicates(subset='item_id', keep='first')
df_detail = df_detail.drop(columns='count_null')

In [ ]:
df_detail.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7730 entries, 0 to 1063
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   item_name            7730 non-null   object
 1   item_brand           7730 non-null   object
 2   item_id              7730 non-null   object
 3   category             7730 non-null   object
 4   xuat_xu_thuong_hieu  7699 non-null   object
 5   gioi_tinh            6273 non-null   object
dtypes: object(6)
memory usage: 422.7+ KB


In [ ]:
df_detail.item_id.nunique()

7730

In [ ]:
# df_detail.to_excel(path_detail + "information_full_data.xlsx", index=False, encoding='utf-8', engine='openpyxl')

In [ ]:
# tmp = pd.read_excel(path_detail + "information_full_data.xlsx", dtype={'item_id': 'str'})
# tmp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7730 entries, 0 to 7729
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   item_name            7730 non-null   object
 1   item_brand           7730 non-null   object
 2   item_id              7730 non-null   object
 3   category             7730 non-null   object
 4   xuat_xu_thuong_hieu  7699 non-null   object
 5   gioi_tinh            6273 non-null   object
dtypes: object(6)
memory usage: 362.5+ KB


In [ ]:
# tmp.head()

,item_name,item_brand,item_id,category,xuat_xu_thuong_hieu,gioi_tinh
0,Dung Dịch Vệ Sinh Phụ Nữ Femfresh Cao Cấp Hằng...,Femfresh,422205453,ca_nhan,Anh,Nữ
1,Nước Hoa Nữ Pinker Bell 05 Sunshine Day EDP 30ml,Pinker Bell,422201933,nuoc_hoa,Hàn Quốc,Nữ
2,Nước Hoa Nữ Salvatore Ferragamo Signorina Ribe...,Salvatore Ferragamo,422206565,nuoc_hoa,Ý,Nữ
3,Nước Hoa Nữ De Memoria #04 Dresden Hương Sành ...,De Memoria,422204136,nuoc_hoa,Hàn Quốc,Nữ
4,Nước Hoa Nam Diamond Homme Dark Blue 45ml (Xanh),Diamond,422210931,nuoc_hoa,Singapore,Nam


## Fix lại cột item_name

In [327]:
path_detail = "C:/Users/lydtm/Downloads/Hasaki/data_processing/data_detail/"
df_detail = pd.read_excel(path_detail + "information_full_data.xlsx", dtype={'item_id': 'str'})

In [328]:
df_detail

,item_name,item_brand,item_id,category,xuat_xu_thuong_hieu,gioi_tinh
0,Dung Dịch Vệ Sinh Phụ Nữ Femfresh Cao Cấp Hằng...,Femfresh,422205453,ca_nhan,Anh,Nữ
1,Nước Hoa Nữ Pinker Bell 05 Sunshine Day EDP 30ml,Pinker Bell,422201933,nuoc_hoa,Hàn Quốc,Nữ
2,Nước Hoa Nữ Salvatore Ferragamo Signorina Ribe...,Salvatore Ferragamo,422206565,nuoc_hoa,Ý,Nữ
3,Nước Hoa Nữ De Memoria #04 Dresden Hương Sành ...,De Memoria,422204136,nuoc_hoa,Hàn Quốc,Nữ
4,Nước Hoa Nam Diamond Homme Dark Blue 45ml (Xanh),Diamond,422210931,nuoc_hoa,Singapore,Nam
...,...,...,...,...,...,...
7725,Dầu Gội Ducray Giúp Ngăn Rụng Tóc 200ml,Ducray,329700001,toc,Pháp,NaN
7726,Dầu Gội OGX Keratin Vào Nếp Suôn Mượt 385ml,OGX,422200029,toc,Mỹ,NaN
7727,Kem Nền Pretty By Flormar Cho Da Dầu 004 Ivory...,Pretty by Flormar,318700043,trang_diem,Thổ Nhĩ Kỳ,NaN
7728,[HSD 02/2024] Kem Dưỡng Cocoon Dưỡng Ẩm Dạng T...,Cocoon,422214587,da_mat,NaN,NaN


In [329]:
# Check
df_detail.groupby('item_id').filter(lambda x: x['item_brand'].nunique() > 1)#.drop_duplicates(subset=['item_brand', 'item_id'])

,item_name,item_brand,item_id,category,xuat_xu_thuong_hieu,gioi_tinh


In [334]:
# Check
df_detail.groupby('item_name').filter(lambda x: x['item_id'].nunique() > 1).drop_duplicates(subset=['item_name', 'item_id']) # bỏ drop 
# tmp['item_id'].isin(df_full['item_id']).sum()

,item_name,item_brand,item_id,category,xuat_xu_thuong_hieu,gioi_tinh


In [335]:
df_detail.item_brand.nunique()

506

In [336]:
# Check
df_full.groupby('item_id').filter(lambda x: x['item_name'].nunique() > 1).drop_duplicates(subset=['item_name', 'item_id'])

,item_name,item_brand,item_id,item_product,item_price,item_bought,item_discount,item_price_previous,item_star,item_comment_number,date,category,bought_by_day


In [333]:
merged_df = pd.merge(df_detail, df_full[['item_id', 'item_name']].drop_duplicates(), on='item_id', how='left')
df_detail['item_name'] = merged_df['item_name_y']

In [337]:
# df_detail.to_excel(path_detail + "information_full_data.xlsx", index=False, encoding='utf-8', engine='openpyxl')

# Đẩy dữ liệu lên DB

In [338]:
## APPLY CATEGORY
from sqlalchemy import create_engine # pip install SQLAlchemy
from sqlalchemy.engine import URL
import pypyodbc # pip install pypyodbc
import pandas as pd # pip install pandas
import urllib.parse
import pyodbc

SERVER_NAME = 'LYDTM\SQLEXPRESS'
DATABASE_NAME = 'HASAKI'
TABLE_NAME = 'CATEGORY'

# ----------------------------------
server = 'LYDTM\SQLEXPRESS'
db = 'HASAKI'

# create Connection and Cursor objects
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + db + ';Trusted_Connection=yes')
cursor = conn.cursor()

# # Read Excel file into a pandas DataFrame
# df = pd.read_excel(path_category + "result_full_data.xlsx", sheet_name="Sheet1", dtype={'item_id': 'str'})

# Define the SQL query to create the table
query1 = """
CREATE TABLE [HASAKI].[dbo].[CATEGORY] (
    item_name nvarchar(255),
    item_brand nvarchar(255),
    item_id nvarchar(255),
    item_product integer,
    item_price float,
    item_bought integer,
    item_discount float,
    item_price_previous float,
    item_star float,
    item_comment_number integer,
    date date,
    category nvarchar(255),
    bought_by_day float
)"""

# Define the SQL query to insert data into the table
query = """
INSERT INTO [HASAKI].[dbo].[CATEGORY] (
    item_name,
    item_brand,
    item_id,
    item_product,
    item_price,
    item_bought,
    item_discount,
    item_price_previous,
    item_star,
    item_comment_number,
    date,
    category,
    bought_by_day
) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"""

# Execute create table query
try:
    cursor.execute(query1)
    conn.commit()
except pyodbc.ProgrammingError:
    pass

# Grab existing row count in the database for validation later
cursor.execute("SELECT count(*) FROM [HASAKI].[dbo].[CATEGORY]")
before_import = cursor.fetchone()

# -----------------------------

path_category = "C:/Users/lydtm/Downloads/Hasaki/data_processing/data_category/"
excel_file_path = path_category + 'result_full_data.xlsx'

connection_string = f"""
    DRIVER={{ODBC Driver 17 for SQL Server}};
    SERVER={SERVER_NAME};
    DATABASE={DATABASE_NAME};
    Trusted_Connection=yes;
    CHARSET=UTF8;
"""

engine = create_engine(f'mssql+pyodbc:///?odbc_connect={urllib.parse.quote_plus(connection_string)}')

excel_file = pd.read_excel(excel_file_path, sheet_name=None, dtype={'item_id': 'str'})
for sheet_name, df_data in excel_file.items():
    print(f'Loading worksheet {sheet_name}...')
    df_data.to_sql(TABLE_NAME, engine, if_exists='append', index=False)

Loading worksheet Sheet1...


In [339]:
## APPLY QUANTITY
from sqlalchemy import create_engine # pip install SQLAlchemy
from sqlalchemy.engine import URL
import pypyodbc # pip install pypyodbc
import pandas as pd # pip install pandas
import urllib.parse
import pyodbc

SERVER_NAME = 'LYDTM\SQLEXPRESS'
DATABASE_NAME = 'HASAKI'
TABLE_NAME = 'QUANTITY'

# ----------------------------------
server = 'LYDTM\SQLEXPRESS'
db = 'HASAKI'

# create Connection and Cursor objects
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + db + ';Trusted_Connection=yes')
cursor = conn.cursor()

# # Read Excel file into a pandas DataFrame
# df = pd.read_excel(path_category + "quantity.xlsx", sheet_name="Sheet1", dtype={'item_id': 'str'})

# Define the SQL query to create the table
query1 = """
CREATE TABLE [HASAKI].[dbo].[QUANTITY] (
    item_id nvarchar(255),
    item_name nvarchar(255),
    quantity integer
)"""

# Define the SQL query to insert data into the table
query = """
INSERT INTO [HASAKI].[dbo].[QUANTITY] (
    item_id,
    item_name,
    quantity
) VALUES (?, ?)"""

# Execute create table query
try:
    cursor.execute(query1)
    conn.commit()
except pyodbc.ProgrammingError:
    pass

# Grab existing row count in the database for validation later
cursor.execute("SELECT count(*) FROM [HASAKI].[dbo].[QUANTITY]")
before_import = cursor.fetchone()

# -----------------------------

path_category = "C:/Users/lydtm/Downloads/Hasaki/data_processing/data_category/"
excel_file_path = path_category + 'quantity.xlsx'

connection_string = f"""
    DRIVER={{ODBC Driver 17 for SQL Server}};
    SERVER={SERVER_NAME};
    DATABASE={DATABASE_NAME};
    Trusted_Connection=yes;
    CHARSET=UTF8;
"""

engine = create_engine(f'mssql+pyodbc:///?odbc_connect={urllib.parse.quote_plus(connection_string)}')

excel_file = pd.read_excel(excel_file_path, sheet_name=None, dtype={'item_id': 'str'})
for sheet_name, df_data in excel_file.items():
    print(f'Loading worksheet {sheet_name}...')
    df_data.to_sql(TABLE_NAME, engine, if_exists='append', index=False)

Loading worksheet Sheet1...


In [340]:
## APPLY DETAIL
from sqlalchemy import create_engine # pip install SQLAlchemy
from sqlalchemy.engine import URL
import pypyodbc # pip install pypyodbc
import pandas as pd # pip install pandas
import urllib.parse
import pyodbc

SERVER_NAME = 'LYDTM\SQLEXPRESS'
DATABASE_NAME = 'HASAKI'
TABLE_NAME = 'INFORMATION'

# ----------------------------------
server = 'LYDTM\SQLEXPRESS'
db = 'HASAKI'

# create Connection and Cursor objects
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + db + ';Trusted_Connection=yes')
cursor = conn.cursor()

# # Read Excel file into a pandas DataFrame
# df = pd.read_excel(path_detail + "information_full_data.xlsx", sheet_name="Sheet1", dtype={'item_id': 'str'})

# Define the SQL query to create the table
query1 = """
CREATE TABLE [HASAKI].[dbo].[INFORMATION] (
    item_name nvarchar(255),
    item_brand nvarchar(255),
    item_id nvarchar(255),
    category nvarchar(255),
    xuat_xu_thuong_hieu nvarchar(255),
    gioi_tinh nvarchar(255)
)"""

# Define the SQL query to insert data into the table
query = """
INSERT INTO [HASAKI].[dbo].[INFORMATION] (
    item_name,
    item_brand,
    item_id,
    category,
    xuat_xu_thuong_hieu,
    gioi_tinh,
) VALUES (?, ?, ?, ?, ?, ?)"""

# Execute create table query
try:
    cursor.execute(query1)
    conn.commit()
except pyodbc.ProgrammingError:
    pass

# Grab existing row count in the database for validation later
cursor.execute("SELECT count(*) FROM [HASAKI].[dbo].[INFORMATION]")
before_import = cursor.fetchone()

# -----------------------------

path_detail = "C:/Users/lydtm/Downloads/Hasaki/data_processing/data_detail/"
excel_file_path = path_detail + 'information_full_data.xlsx'

connection_string = f"""
    DRIVER={{ODBC Driver 17 for SQL Server}};
    SERVER={SERVER_NAME};
    DATABASE={DATABASE_NAME};
    Trusted_Connection=yes;
    CHARSET=UTF8;
"""

engine = create_engine(f'mssql+pyodbc:///?odbc_connect={urllib.parse.quote_plus(connection_string)}')

excel_file = pd.read_excel(excel_file_path, sheet_name=None, dtype={'item_id': 'str'})
for sheet_name, df_data in excel_file.items():
    print(f'Loading worksheet {sheet_name}...')
    df_data.to_sql(TABLE_NAME, engine, if_exists='append', index=False)

Loading worksheet Sheet1...
